## 당뇨병 여부 판단
- 이상치 처리 (Glucose, BloodPressure, SkinThickness, Insulin, BMI가 0인 값)

## [참고]작업형2 문구
- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

### 데이터 파일 읽기 예제
- import pandas as pd
- X_test = pd.read_csv("data/X_test.csv")
- X_train = pd.read_csv("data/X_train.csv")
- y_train = pd.read_csv("data/y_train.csv")

### 사용자 코딩

### 답안 제출 참고
- 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
- pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

In [21]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Outcome')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 9), (154, 9), (614, 2), (154, 2))

In [22]:
display(X_train)
display(y_train)

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
147,147,2,106,64,35,119,30.5,1.400,34
344,344,8,95,72,0,0,36.8,0.485,57
390,390,1,100,66,29,196,32.0,0.444,42
150,150,1,136,74,50,204,37.4,0.399,24
132,132,3,170,64,37,225,34.5,0.356,30
...,...,...,...,...,...,...,...,...,...
621,621,2,92,76,20,0,24.2,1.698,28
128,128,1,117,88,24,145,34.5,0.403,40
57,57,0,100,88,60,110,46.8,0.962,31
341,341,1,95,74,21,73,25.9,0.673,36


,id,Outcome
147,147,0
344,344,0
390,390,0
150,150,0
132,132,1
...,...,...
621,621,0
128,128,1
57,57,0
341,341,0


In [23]:
# 데이터가 0인 값의 수 확인
print(X_train.Glucose[X_train.Glucose==0].value_counts())
print(X_train.BloodPressure[X_train.BloodPressure==0].value_counts())
print(X_train.SkinThickness[X_train.SkinThickness==0].value_counts())
print(X_train.Insulin[X_train.Insulin==0].value_counts())
print(X_train.BMI[X_train.BMI==0].value_counts())

0    5
Name: Glucose, dtype: int64
0    31
Name: BloodPressure, dtype: int64
0    175
Name: SkinThickness, dtype: int64
0    287
Name: Insulin, dtype: int64
0.0    9
Name: BMI, dtype: int64


In [24]:
# 데이터가 0인 값을 평균으로 변경
cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for i in cols:
    X_train[i].replace(0, X_train[i].mean(), inplace=True)
    X_test[i].replace(0, X_train[i].mean(), inplace=True)

In [25]:
# id 컬럼 제거
X_train_drop = X_train.drop(columns='id')
X_test_drop = X_test.drop(columns='id')

In [26]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X_train_scaled = ss.fit_transform(X_train_drop)
X_test_scaled = ss.transform(X_test_drop)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

y = y_train['Outcome']

x_train, x_val, Y_train, Y_val = train_test_split(X_train_scaled, y, test_size=0.2, random_state=99)

rf = RandomForestClassifier()
rf.fit(x_train, Y_train)

RandomForestClassifier()

In [28]:
# 랜덤 포레스트 모델 성능 확인
print(accuracy_score(Y_train, rf.predict(x_train)))
# 검증
print(accuracy_score(Y_val, rf.predict(x_val)))

1.0
0.8536585365853658


In [29]:
pd.DataFrame({'id':X_test.id, 'Outcome':rf.predict(X_test_drop)}).to_csv('000000000.csv', index=False)

In [30]:
result = pd.read_csv('000000000.csv')
result

,id,Outcome
0,258,1
1,220,1
2,438,1
3,130,1
4,730,1
...,...,...
149,678,1
150,500,1
151,256,1
152,80,1


In [32]:
round(rf.score(X_test_scaled, y_test['Outcome']) * 100, 2)

74.68

# 풀이
---

## Start

## 라이브러리 및 데이터 불러오기

In [2]:
# 라이브러리
import pandas as pd

In [3]:
# 데이터 불러오기 (생략)
X_train.shape, y_train.shape, X_test.shape

((614, 9), (614, 2), (154, 9))

## EDA

In [4]:
X_train.head()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
147,147,2,106,64,35,119,30.5,1.400,34
344,344,8,95,72,0,0,36.8,0.485,57
390,390,1,100,66,29,196,32.0,0.444,42
150,150,1,136,74,50,204,37.4,0.399,24
132,132,3,170,64,37,225,34.5,0.356,30


In [5]:
y_train.value_counts()

id   Outcome
0    1          1
514  0          1
504  0          1
506  1          1
507  0          1
               ..
265  0          1
266  1          1
267  0          1
268  0          1
766  1          1
Length: 614, dtype: int64

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 614 entries, 147 to 116
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        614 non-null    int64  
 1   Pregnancies               614 non-null    int64  
 2   Glucose                   614 non-null    int64  
 3   BloodPressure             614 non-null    int64  
 4   SkinThickness             614 non-null    int64  
 5   Insulin                   614 non-null    int64  
 6   BMI                       614 non-null    float64
 7   DiabetesPedigreeFunction  614 non-null    float64
 8   Age                       614 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 48.0 KB


In [7]:
X_train.isnull().sum()

id                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [8]:
X_test.isnull().sum()

id                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [9]:
X_train.describe()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,386.978827,3.845277,120.705212,68.530945,20.982085,82.229642,31.948534,0.477471,33.319218
std,219.881344,3.388952,32.047977,19.761915,15.965031,116.525772,7.734385,0.339811,11.940953
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,196.500000,1.000000,99.000000,62.000000,0.000000,0.000000,27.400000,0.244250,24.000000
50%,388.000000,3.000000,117.000000,72.000000,23.000000,44.000000,32.250000,0.377000,29.000000
75%,575.750000,6.000000,140.000000,80.000000,33.000000,130.000000,36.600000,0.628500,41.000000
max,766.000000,17.000000,199.000000,114.000000,99.000000,846.000000,59.400000,2.420000,81.000000


## 데이터 전처리

In [10]:
#이상치 처리
#Train
print('Glucose:',len(X_train[X_train['Glucose']==0]))
print('BloodPressure:',len(X_train[X_train['BloodPressure']==0]))
print('SkinThickness:',len(X_train[X_train['SkinThickness']==0]))
print('Insulin:',len(X_train[X_train['Insulin']==0]))
print('BMI:',len(X_train[X_train['BMI']==0]))

Glucose: 5
BloodPressure: 31
SkinThickness: 175
Insulin: 287
BMI: 9


In [11]:
#Test
print('Glucose:',len(X_test[X_test['Glucose']==0]))
print('BloodPressure:',len(X_test[X_test['BloodPressure']==0]))
print('SkinThickness:',len(X_test[X_test['SkinThickness']==0]))
print('Insulin:',len(X_test[X_test['Insulin']==0]))
print('BMI:',len(X_test[X_test['BMI']==0]))

Glucose: 0
BloodPressure: 4
SkinThickness: 52
Insulin: 87
BMI: 2


트레인에만 0이 있는 포도당(Glucose)는 삭제
나머지는 평균값으로 대체 하여 이상치를 처리함

In [12]:
# 포도당 이상치 삭제
del_idx = X_train[(X_train['Glucose']==0)].index
del_idx

Int64Index([182, 75, 342, 502, 349], dtype='int64')

In [13]:
print('Glucose 이상치 삭제 전 :', X_train.shape, y_train.shape)
X_train = X_train.drop(index=del_idx, axis=0)
y_train = y_train.drop(index=del_idx, axis=0)
print('Glucose 이상치 삭제 후 :', X_train.shape, y_train.shape)

Glucose 이상치 삭제 전 : (614, 9) (614, 2)
Glucose 이상치 삭제 후 : (609, 9) (609, 2)


In [14]:
# 포도당을 제외한 이상치, 평균값으로 대체
cols = ['BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
cols_mean = X_train[cols].mean()
X_train[cols].replace(0, cols_mean)

,BloodPressure,SkinThickness,Insulin,BMI
147,64.0,35.00000,119.000000,30.5
344,72.0,20.91133,82.866995,36.8
390,66.0,29.00000,196.000000,32.0
150,74.0,50.00000,204.000000,37.4
132,64.0,37.00000,225.000000,34.5
...,...,...,...,...
621,76.0,20.00000,82.866995,24.2
128,88.0,24.00000,145.000000,34.5
57,88.0,60.00000,110.000000,46.8
341,74.0,21.00000,73.000000,25.9


In [15]:
# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.fit_transform(X_test[cols])

In [16]:
# id 제외
X = X_train.drop('id',axis=1)
test = X_test.drop('id',axis=1)

In [17]:
from sklearn.svm import SVC
model = SVC(random_state=2022)
model.fit(X, y_train['Outcome'])
predictions = model.predict(test)

In [18]:
# 오버피팅 되었을 경우에 점수가 잘나올 수 있음 (객관적인 평가 아님, 밸리데이션 데이터로 평가 필요함)
round(model.score(X, y_train['Outcome']) * 100, 2) 

83.42

In [19]:
output = pd.DataFrame({'idx': X_test.index, 'Outcome': predictions})
output.head()

,idx,Outcome
0,258,1
1,220,1
2,438,0
3,130,1
4,730,0


In [ ]:
# 수험번호.csv로 출력
output.to_csv('1234567.csv', index=False)

## 결과 체점 (수험자는 알 수 없는 부분임)

In [20]:
round(model.score(test, y_test['Outcome']) * 100, 2) # 60점대로 낮은 정확도를 보여줌

76.62